# **AstrID:**  *model training*

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.preprocessing import image
from keras.initializers import he_uniform
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.optimizers import Adam

# Import custom function to create and train the U-Net model
from models.unet import unet_model

import os
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
from astropy.io import fits

# Import custom functions to extract Image arrays and Pixel Mask arrays from our created fits files dataset
from dataGathering import extractImageArray, extractPixelMaskArray, extract_star_catalog
from dataGathering import getStarData, getImagePlot, getPixelMaskPlot
from dataGathering import displayRawImage, displayRawPixelMask, displayImagePlot, displayPixelMaskPlot, displayPixelMaskOverlayPlot

# Import custom functions to preprocess Image and Pixel Mask arrays
from imageProcessing import normalizeImages, stackImages, stackMasks, preprocessImage

# Import custom logging function
from log import write_to_log


2024-11-18 00:55:57.278562: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-18 00:55:58.572036: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-11-18 00:56:02.129427: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-18 00:56:02.270407: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-18 00:56:02.270514: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("GPU is available")
    
    # Clear GPU memory
    K.clear_session()
    tf.config.experimental.reset_memory_stats('GPU:0')

GPU is available


2024-11-18 00:56:02.298634: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-18 00:56:02.298769: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-18 00:56:02.298816: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-18 00:56:02.539189: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-18 00:56:02.539489: I tensorflow/compile

In [4]:
# getStarData('II/246', 250, 'data')

In [5]:
# Create images and masks arrays lists
images = []
masks = []

# Create df to store the star data inside each fits file
star_data = []

# Create a list of all the fits files in the dataset folder
fits_files = os.listdir('data/fits/')

# For all the fits files in the dataset folder specified in file_path, extract the image and mask arrays to the respective lists
file_path = 'data/fits/'
# for file in os.listdir(file_path):
for file in os.listdir(file_path):
    if file.endswith('.png'):
        os.remove(file_path + file)
    if file.startswith('data') and file.endswith('.fits'):
        images.append(extractImageArray(file_path + file))
        masks.append(extractPixelMaskArray(file_path + file))
        star_data.append(extract_star_catalog(file_path + file))

        print(file + ' added to dataset')

data47.fits added to dataset
data8.fits added to dataset
data93.fits added to dataset
data202.fits added to dataset
data79.fits added to dataset
data116.fits added to dataset
data187.fits added to dataset
data100.fits added to dataset
data10.fits added to dataset
data91.fits added to dataset
data34.fits added to dataset
data117.fits added to dataset
data148.fits added to dataset
data107.fits added to dataset
data38.fits added to dataset
data134.fits added to dataset
data238.fits added to dataset
data136.fits added to dataset
data5.fits added to dataset
data189.fits added to dataset
data33.fits added to dataset
data223.fits added to dataset
data162.fits added to dataset
data13.fits added to dataset
data28.fits added to dataset
data208.fits added to dataset
data118.fits added to dataset
data197.fits added to dataset
data143.fits added to dataset
data178.fits added to dataset
data102.fits added to dataset
data200.fits added to dataset
data129.fits added to dataset
data174.fits added to da

# Prepare the data for training

### Convert to 3-Channel Images

The images we have are 512 x 512 pixels, but our model requires them to be in the shape `(512, 512, 3)`, similar to standard RGB images. To achieve this, we stack the single-channel images along the last axis three times, converting them into 3-channel images. This transformation is necessary because the model typically expects 3-channel input images.

For the masks, the model expects them to be in the shape `(512, 512, 1)`. Therefore, we expand the masks along the last axis to add a new dimension, ensuring they have the correct shape.

Additionally, both the images and masks need to be converted to NumPy arrays, as this is the desired format for the training model. Below, we perform these conversions to ensure the data is in the correct format for training.

Notice when displaying the shape of the `train_images` list below we see it is an array of 250 images of the shape mentioned above, giving us a shape `(250, 512, 512, 3)`

In [6]:
stacked_images = stackImages(images)
stacked_masks = stackMasks(masks)

### Normalize the Images

To standardize the pixel values in our images, we need to normalize them to a common range.
We will use min-max normalization to scale the pixel values to a range between 0.0 and 1.0.

In [7]:
# normalized_images = normalizeImages(stacked_images)

###  Prepare the model for training


In [8]:
#############################################

# Define the base exponent - Change this number to increase or decrease the number of filters by a power of 2
base_exponent = 5
#############################################
# base_exponent = 5, so filters result in [32, 64, 128, 256, 512], this was the original value
# base_exponent = 6, so filters result in [64, 128, 256, 512, 1024]
# base_exponent = 7, so filters result in [128, 256, 512, 1024, 2048]
# base_exponent = 8, so filters result in [256, 512, 1024, 2048, 4096]
# base_exponent = 9, so filters result in [512, 1024, 2048, 4096, 8192]
# base_exponent = 10, so filters result in [1024, 2048, 4096, 8192, 16384]

# Generate the filters based on powers of 2
filters = [2 ** (base_exponent + i) for i in range(5)]
#############################################
# This list comprehension generates the filter sizes by raising 2 to the powers starting from the base exponent and increasing by 1 for each subsequent filter.
#############################################


In [9]:
filters

[32, 64, 128, 256, 512]

In [10]:


# Define hyperparameters
hyperparameters = {
    'input_shape': (512, 512, 3),
    'filters': filters,
    'kernel_size': (3, 3),
    'activation': 'relu',
    'padding': 'same',
    # 'initializer': he_uniform(),
    'initializer': he_uniform,
    # 'optimizer': 'adam',
    'optimizer': Adam(learning_rate=0.00005),
    # 'loss': weightedBinaryCrossEntropy,
    'loss': 'binary_crossentropy',
    'weights' : {0 : 1.0, 1 : 5.0},
    'metrics': ['accuracy'],
    'epochs': 100,
    'batch_size': 4,
    'early_stopping_patience': 10,
    'test_size': 0.2,
    'random_state': 0,
    'seed': 42
}

# Create and compile the model using hyperparameters
model = unet_model(
    input_shape=hyperparameters['input_shape'],
    filters=hyperparameters['filters'],
    kernel_size=hyperparameters['kernel_size'],
    activation=hyperparameters['activation'],
    padding=hyperparameters['padding'],
    initializer=hyperparameters['initializer']
)
model.compile(
    optimizer=hyperparameters['optimizer'],
    loss=hyperparameters['loss'],
    metrics=hyperparameters['metrics']
)

**Create Generators for Training and Validation**:
   - Use the `flow` method to create generators for training and validation images and masks.
   - Ensure that the `seed` parameter is the same for both image and mask generators to maintain alignment.

 **Custom Generator**:
   - Define a custom generator function `custom_generator` that yields a tuple (images, masks, sample_weights).

In [11]:
# Split the dataset into training and validation sets
train_images, val_images, train_masks, val_masks = train_test_split(stacked_images, stacked_masks, test_size=hyperparameters['test_size'], random_state=hyperparameters['random_state'])

In [12]:
# Show distribution of training and validation sets
print('Training images: ', train_images.shape)
print('Training masks: ', train_masks.shape)
print('Validation images: ', val_images.shape)
print('Validation masks: ', val_masks.shape)

Training images:  (200, 512, 512, 3)
Training masks:  (200, 512, 512, 1)
Validation images:  (50, 512, 512, 3)
Validation masks:  (50, 512, 512, 1)


# Train the model

In [ ]:
from sklearn.utils import class_weight

# Implement Early stopping to cut useless epochs
early_stopping = EarlyStopping(
    monitor='val_loss',  # or another metric like 'val_accuracy'
    patience=hyperparameters['early_stopping_patience'],         # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restores the model to the best state after stopping
)


# Train the model
history = model.fit(
    train_images, train_masks, 
    validation_data=(val_images, val_masks), 
    epochs=hyperparameters['epochs'], 
    batch_size=hyperparameters['batch_size'],
    callbacks=[early_stopping],
    class_weight=hyperparameters['weights']
)

Epoch 1/100


2024-11-18 00:56:26.342012: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-11-18 00:56:28.378463: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa5edec3980 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-18 00:56:28.378504: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-11-18 00:56:28.401019: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-18 00:56:28.638650: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 [==============================] - 21s 123ms/step - loss: 66.2607 - accuracy: 0.9618 - val_loss: 12.2195 - val_accuracy: 0.9794
Epoch 2/100
50/50 [==============================] - 7s 147ms/step - loss: 31.6754 - accuracy: 0.9687 - val_loss: 7.7368 - val_accuracy: 0.9792
Epoch 3/100
50/50 [==============================] - 7s 138ms/step - loss: 25.1597 - accuracy: 0.9671 - val_loss: 6.8003 - val_accuracy: 0.9726
Epoch 4/100
50/50 [==============================] - 7s 141ms/step - loss: 21.1410 - accuracy: 0.9683 - val_loss: 6.5929 - val_accuracy: 0.9673
Epoch 5/100
50/50 [==============================] - 7s 145ms/step - loss: 18.8424 - accuracy: 0.9665 - val_loss: 6.7979 - val_accuracy: 0.9555
Epoch 6/100
50/50 [==============================] - 5s 96ms/step - loss: 17.0350 - accuracy: 0.9673 - val_loss: 4.7386 - val_accuracy: 0.9743
Epoch 7/100
50/50 [==============================] - 7s 141ms/step - loss: 15.4267 - accuracy: 0.9664 - val_loss: 3.8722 - val_accuracy: 0.9839
Epo

In [ ]:
import getpass

user = getpass.getuser()

# Save the model
saved_models_path = 'models/saved_models/'
training_size = str(len(train_images))
saved_model_name = datetime.datetime.now().strftime("%Y_%m_%d-%H%M_") + training_size + '_unet_model_' + user +  '.keras'
model.save(saved_models_path + saved_model_name)

#log model parameters, time, and user
write_to_log(history, hyperparameters, saved_model_name)

In [ ]:
# Show training loss and validation loss
print('Training loss: ', history.history['loss'][-1])
print('Validation loss: ', history.history['val_loss'][-1])

# Show training accuracy and validation accuracy
print('Training accuracy: ', history.history['accuracy'][-1])
print('Validation accuracy: ', history.history['val_accuracy'][-1])

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(24, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
# plt.savefig('results/figures/' + saved_model_name.removesuffix('unet_model.keras') + '_training_validation.png')
plt.show()

In [ ]:
# Set a larger figure size
plt.figure(figsize=(10, 6))

# Plot the training and validation loss
plt.plot(history.history['loss'], color='blue', linestyle='-', linewidth=2, label='Training Loss')
plt.plot(history.history['val_loss'], color='orange', linestyle='--', linewidth=2, label='Validation Loss')

# Add title and labels
plt.title('Training and Validation Loss Over Epochs', fontsize=16)
plt.xlabel('Epochs', fontsize=14)
plt.ylabel('Loss', fontsize=14)

# Add a grid for better readability
plt.grid(True, which='both', linestyle='--', linewidth=0.5)

# Add legend to differentiate between training and validation loss
plt.legend(loc='best', fontsize=12)

# Set limits for better visualization
plt.xlim(0, len(history.history['loss']) - 1)  # From epoch 0 to the last epoch
plt.ylim(min(history.history['loss']) * 0.95, max(history.history['val_loss']) * 1.05)

# Show the plot
plt.tight_layout()
# plt.savefig('results/figures/' + saved_model_name.removesuffix('unet_model.keras') + '_training_validation_loss.png')
plt.show()
